In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import nltk
from pydub import AudioSegment
import openai
from pathlib import Path

In [2]:
# Read the file
with open('.\data\Autobiography of Ben Franklin.txt', 'r') as file:
    data = file.read()

# Remove unnecessary line breaks
data = data.replace('\n', ' ')
data = data.replace('\r', ' ')

# Remove extra spaces
import re
data = re.sub(' +', ' ', data)
data = re.sub(r'\[.*?\]', '', data)

# Write the cleaned data back to the file
with open('file.txt', 'w') as file:
    file.write(data)

In [12]:
#WIP to parse out a set number of chapters to make it easier to listen to the audio book
num_chapters = 4
chapter_size = len(data) // num_chapters
chapters = [data[i:i + chapter_size] for i in range(0, len(data), chapter_size)]

In [5]:
# Read the text file
with open('file.txt', 'r') as file:
    text = file.read()

# Tokenize the text and split it into chunks of 4096 tokens
chunk_size = 4096
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Initialize an empty audio segment
combined = AudioSegment.empty()

In [15]:
# Loop over the first two chunks to test before doing the whole text!
for i, chunk in enumerate(chunks[:2]):
    # Send the chunk to the OpenAI API
    response = openai.audio.speech.create(
        model="tts-1-hd",
        voice="onyx",
        input=chunk,
        speed=1.5
    )

    # Save the response to a temporary file
    temp_file_path = Path(f'tmp{i}.mp3')
    response.stream_to_file(temp_file_path)

    # Append the audio from the temporary file to the combined audio
    combined += AudioSegment.from_mp3(temp_file_path)

# Save the combined audio to a file
combined.export("output.mp3", format="mp3")

<_io.BufferedRandom name='output.mp3'>

In [16]:
# Loop over the chunks (whole text doc)
for i, chunk in enumerate(chunks):
    # Send the chunk to the OpenAI API
    response = openai.audio.speech.create(
        model="tts-1-hd",
        voice="onyx",
        input=chunk,
        speed=1.25
    )

    # Save the response to a temporary file
    temp_file_path = Path(f'temp{i}.mp3')
    response.stream_to_file(temp_file_path)

    # Append the audio from the temporary file to the combined audio
    combined += AudioSegment.from_mp3(temp_file_path)

# Save the combined audio to a file
combined.export("Autobiography Benjamin Franklin.mp3", format="mp3")

<_io.BufferedRandom name='Autobiography Benjamin Franklin.mp3'>

In [6]:
#clean up the temp files
for i, chunk in enumerate(chunks):
    temp_file_path = Path(f'temp{i}.mp3')
    os.remove(temp_file_path)